In [1]:
import numpy as np
import pandas as pd
from math import *

from coeffs import ResponseType, CoeffsMinPhase, CoeffsType1Pad, CoeffsType2, CoeffsHalfBand
from filters import SsrFilter, FfaMcmFilter, PolyMcmFilter

In [ ]:
coeff_constrs = [
    lambda taps : CoeffsMinPhase(ResponseType.LP, [0,0.3,0.4,1.0], taps, 16),
    lambda taps : CoeffsType1Pad(ResponseType.LP, [0,0.3,0.4,1.0], taps, 16),
    lambda taps :    CoeffsType2(ResponseType.LP, [0,0.3,0.4,1.0], taps, 16),
    lambda taps : CoeffsHalfBand(taps, 16)
]

fir_constrs = [
    lambda par, taps, ws, out_dir :     SsrFilter(par, 16, ws, 775.0, out_dir),
    lambda par, taps, ws, out_dir : PolyMcmFilter(par, 16, ws, 775.0, out_dir),
    lambda par, taps, ws, out_dir :  FfaMcmFilter(par, 16, ws, 775.0, out_dir),
]

results = []

for par in [2,4,8,16]:
    for taps in [2,4,8,16,32,64,128]:
        if par > taps:
            continue
        for f_coef in coeff_constrs:
            for f_fir in fir_constrs:
                ws = f_coef(taps)
                fir = f_fir(par, taps, ws, './outputs')
                res = fir.impl()
                print(res)
                results.append(res)

{'group': './outputs', 'name': 'Ssr_P2_IB16_FCLK775.0_WS_N2_CB16_minphase_LP_F0-0.3-0.4-1.0', 'structure': 'Ssr', 'in_width': 16, 'c_width': 16, 'parallelism': 2, 'fclk_actual': 1029.8661174047372, 'taps': 2, 'luts': 26, 'regs': 155, 'clbs': 16, 'dsps': 4}
{'group': './outputs', 'name': 'PolyMcm_P2_IB16_FCLK775.0_WS_N2_CB16_minphase_LP_F0-0.3-0.4-1.0', 'structure': 'PolyMcm', 'in_width': 16, 'c_width': 16, 'parallelism': 2, 'fclk_actual': 816.9934640522875, 'taps': 2, 'luts': 306, 'regs': 605, 'clbs': 66, 'dsps': 0}
{'group': './outputs', 'name': 'FfaMcm_P2_IB16_FCLK775.0_WS_N2_CB16_minphase_LP_F0-0.3-0.4-1.0', 'structure': 'FfaMcm', 'in_width': 16, 'c_width': 16, 'parallelism': 2, 'fclk_actual': 586.8544600938967, 'taps': 2, 'luts': 409, 'regs': 654, 'clbs': 73, 'dsps': 0}
{'group': './outputs', 'name': 'Ssr_P2_IB16_FCLK775.0_WS_N2_CB16_type1p_LP_F0-0.3-0.4-1.0', 'structure': 'Ssr', 'in_width': 16, 'c_width': 16, 'parallelism': 2, 'fclk_actual': 977.5171065493645, 'taps': 2, 'luts': 2

In [ ]:
import plotly.express as px
display(px.line(df, x='taps', facet_col='parallelism',y='clbs', color='structure', log_x=True))
display(px.line(df, x='taps', facet_col='parallelism',y='luts', color='structure', log_x=True))
display(px.line(df, x='taps', facet_col='parallelism',y='regs', color='structure', log_x=True))
display(px.line(df, x='taps', facet_col='parallelism',y='dsps', color='structure', log_x=True))

In [83]:
df_export = df.copy()
df_export['dsps'] = df_export['dsps']*100/4272
df_export = df_export.pivot(index=['taps'], columns=['structure','parallelism'], values='dsps')
df_export.columns = ['_'.join([str(c) for c in col]) for col in df_export.columns.values]

df_export.to_csv(f'outputs/dsps_{group}.csv', index = True, header=True)

In [70]:
df_export = df.copy()
df_export['clbs'] = df_export['clbs']*100/53160
df_export = df_export.pivot(index=['taps'], columns=['structure','parallelism'], values='clbs')
df_export.columns = ['_'.join([str(c) for c in col]) for col in df_export.columns.values]

df_export.to_csv(f'outputs/clbs_{group}.csv', index = True, header=True)